In [1]:
import pandas as pd
import json

print("--- ETAPA 1: PRÉ-PROCESSAMENTO E CONSOLIDAÇÃO ---")

# Bloco 1: Processamento do vagas.json
print("\nProcessando vagas.json...")
with open('vagas.json', 'r', encoding='utf-8') as f:
    vagas_data = json.load(f)

vagas_df = pd.DataFrame.from_dict(vagas_data, orient='index')
vagas_df.index.name = 'id_vaga'

info_basicas_df = pd.json_normalize(vagas_df['informacoes_basicas']).add_prefix('info.').set_index(vagas_df.index)
perfil_vaga_df = pd.json_normalize(vagas_df['perfil_vaga']).add_prefix('perfil.').set_index(vagas_df.index)
beneficios_df = pd.json_normalize(vagas_df['beneficios']).add_prefix('beneficios.').set_index(vagas_df.index)

vagas_final_df = pd.concat([
    vagas_df.drop(columns=['informacoes_basicas', 'perfil_vaga', 'beneficios']),
    info_basicas_df,
    perfil_vaga_df,
    beneficios_df
], axis=1).reset_index()
print("vagas.json processado.")

# Bloco 2: Processamento do prospects.json
print("\nProcessando prospects.json...")
with open('prospects.json', 'r', encoding='utf-8') as f:
    prospects_data = json.load(f)

prospects_list = []
for vaga_id, data in prospects_data.items():
    for prospect in data['prospects']:
        prospect_info = {
            'id_vaga': vaga_id,
            'titulo': data.get('titulo'),
            'modalidade': data.get('modalidade'),
            **prospect
        }
        prospects_list.append(prospect_info)

prospects_final_df = pd.DataFrame(prospects_list).add_prefix('prospect.')
prospects_final_df.rename(columns={'prospect.id_vaga': 'id_vaga'}, inplace=True)
print("prospects.json processado.")

# Bloco 3: Processamento do applicants.json
print("\nProcessando applicants.json...")
with open('applicants.json', 'r', encoding='utf-8') as f:
    applicants_data = json.load(f)

applicants_df = pd.DataFrame.from_dict(applicants_data, orient='index')
applicants_df.index.name = 'id_candidato'

nested_columns = [col for col in applicants_df.columns if isinstance(applicants_df[col].dropna().iloc[0], dict)]
df_parts = [applicants_df.drop(columns=nested_columns)]
for col in nested_columns:
    normalized_part = pd.json_normalize(applicants_df[col]).add_prefix(f"{col}.")
    normalized_part.index = applicants_df.index
    df_parts.append(normalized_part)

applicants_final_df = pd.concat(df_parts, axis=1).reset_index()
print("applicants.json processado.")

# Bloco 4: Unificação (Merge) dos DataFrames
print("\nUnificando os dados...")
prospects_final_df['id_vaga'] = prospects_final_df['id_vaga'].astype(str)
vagas_final_df['id_vaga'] = vagas_final_df['id_vaga'].astype(str)
prospects_final_df['prospect.codigo'] = prospects_final_df['prospect.codigo'].astype(str)
applicants_final_df['id_candidato'] = applicants_final_df['id_candidato'].astype(str)

dados_consolidados = pd.merge(prospects_final_df, vagas_final_df, on='id_vaga', how='left')
dados_consolidados = pd.merge(dados_consolidados, applicants_final_df, left_on='prospect.codigo', right_on='id_candidato', how='inner')
print("Unificação concluída.")

# Bloco 5: Criação da Variável Alvo
print("\nCriando a variável-alvo 'match'...")
dados_consolidados['match'] = (dados_consolidados['prospect.situacao_candidado'] == 'Contratado pela Decision').astype(int)
print("Variável-alvo criada.")

# Bloco 6: Análise Final e Salvamento em JSON
print("\n--- BASE DE DADOS CONSOLIDADA ---")
print(f"Dimensões da base final: {dados_consolidados.shape[0]} linhas, {dados_consolidados.shape[1]} colunas")
print("\nDistribuição da variável-alvo:")
print(dados_consolidados['match'].value_counts())

# Salva o resultado para a próxima etapa em formato JSON
# orient='records', lines=True é eficiente para leitura posterior com pandas
dados_consolidados.to_json('dados_consolidados.json', orient='records', lines=True, force_ascii=False)
print("\nArquivo 'dados_consolidados.json' salvo com sucesso!")
print("\n--- ETAPA 1 CONCLUÍDA ---")

--- ETAPA 1: PRÉ-PROCESSAMENTO E CONSOLIDAÇÃO ---

Processando vagas.json...
vagas.json processado.

Processando prospects.json...
prospects.json processado.

Processando applicants.json...
applicants.json processado.

Unificando os dados...
Unificação concluída.

Criando a variável-alvo 'match'...
Variável-alvo criada.

--- BASE DE DADOS CONSOLIDADA ---
Dimensões da base final: 45095 linhas, 113 colunas

Distribuição da variável-alvo:
match
0    42840
1     2255
Name: count, dtype: int64

Arquivo 'dados_consolidados.json' salvo com sucesso!

--- ETAPA 1 CONCLUÍDA ---
